## Introduction

In this project, we are taking a look at what makes an ice cream flavor more enjoyable for people to eat. We found this ice cream rating dataset on Kaggle which rates ice cream flavor of four ice cream producers and wish to understand the underlying reasons which makes an ice cream flavor rating higher than other ones.  Throughout this analysis, we will take a look at different hypotheses related to the type of ingredients contained within one ice cream, toppings used as well as what would be the perfect mix of toppings and ingredients which, could potentially produce the best rated ice cream flavor maximizing the ratings.

## Data Set Description

The dataset we chose is from Kaggle and contains reviews of multiple ice cream flavors across 4 major brands. Reviews comprise star ratings as well as a descriptive text which collected from the brand websites. 

Link: [https://www.kaggle.com/tysonpo/ice-cream-dataset](https://www.kaggle.com/tysonpo/ice-cream-dataset)

- **products.csv**: contains information about each flavor
    - 242 observations
    - Variables: Ice cream key, brand name, name, subhead, description, rating, rating counts, ingredients
- **reviews.csv**: contains reviews for each flavor of ice cream
    - 21,674 observations
    - Variables: Date of the review, star rating, title, helpful review indicator, review text

Additional dataset:
- **dairy.txt**: contains dairy keywords
    - Source: https://www.godairyfree.org/dairy-free-grocery-shopping-guide/dairy-ingredient-list-2

The dataset had to be cleaned in order to facilitate the statistical analysis to be done. This includes extracting the ingredient list, tokenizing it, and creating indicator variables. Additionally, we need to restrict the number of ingredients to keep track of to the following: raspberries, peanuts, almonds, coffee, chocolate, strawberries and raspberries. More will be added to this list following analysis on ingredient prevalence across flavors and brands.

In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
#read csv files

#Data from kaggle:
#https://www.kaggle.com/tysonpo/ice-cream-dataset

df = pd.read_csv("products.csv")
df_reviews = pd.read_csv("reviews.csv")

#dairy free keywords
#https://www.godairyfree.org/dairy-free-grocery-shopping-guide/dairy-ingredient-list-2
dairy = pd.read_csv("dairy.txt", header = None)


## Functions

In [279]:
def split_ingredients(split_regex, ing, ignore_organic = True):
    
    """
    Split ingredients string given specified regex
    Ignores blanks
    Can ignore Organics label (i.e. Milk = Organic Milk), default TRUE
    Returns list of elements
    """
    
    split_list = [x.lstrip() for x in re.split(split_regex, ing) if x != ""]
    
    if ignore_organic:
        return_list = [re.sub("ORGANIC", "", item).lstrip().rstrip() for item in split_list]
    else:
        return_list = split_list
    
    return return_list

def get_ingredients_count(split_regex, df, ignore_organic = True):
    
    """
    Split ingredients series by chr string specified by regex
    Parent- separate by commas, ignore parenthesis 
    Child- separate by commas
    Return dictionary with keys as ingredients, values as frequency
    
    """
    
    ingredient_dict = {}
    ingredient_series = df["ingredients"]
    
    for i in range(len(ingredient_series)):
        ingredient_split = split_ingredients(split_regex, ingredient_series[i], ignore_organic)
        for element in ingredient_split:
            if element in ingredient_dict.keys():
                ingredient_dict[element] = ingredient_dict[element] + 1
            else:
                ingredient_dict[element] = 1
    return ingredient_dict

def get_ingredients(split_regex, df, ignore_organic = True):
    
    """
    Split ingredients series by chr string specified by regex
    Parent- separate by commas, ignore parenthesis 
    Child- separate by commas
    Return dictionary with keys as flavor keys, values as split ingredients
    """
    
    ingredient_dict = {}
    
    for i in range(len(df)):
        ingredient_split = split_ingredients(split_regex, df.iloc[i]["ingredients"], ignore_organic)
        ingredient_dict[df.iloc[i]["key"]] = ingredient_split
        
    return ingredient_dict


def get_indicator_df(split_regex, target_ing, df, match_all = True, ignore_organic = True):
    
    """
    Split ingredients specified by regex
    match_all: whether to match entire word or partial (default: all)
    i.e.:
        all: "Skim Milk" -- match entire Skim Milk
        partial: "Organic" --any ingredient contains word organic
        
    Returns dataFrame with flavors and True/False values for all ingredients in target_ing
    """
    
    ing_dict = get_ingredients(split_regex, df, ignore_organic)
    ing_list = list(get_ingredients_count(split_regex, df, ignore_organic).keys())
    if isinstance(target_ing, str):
        target_ing = [target_ing]

    indicator_dict = {}

    if match_all:
        
        target_ing_df = pd.DataFrame({"ingredients" : target_ing})
        target_ing = [ing.upper() for ing in target_ing if ing.upper() in ing_list]
        for key in ing_dict.keys():
            indicator_dict[key] = [item in ing_dict[key] for item in target_ing_df["ingredients"]]
            indicator_df = target_ing_df.join(pd.DataFrame(data = indicator_dict)).set_index("ingredients").transpose()
    
    else:
        for key in ing_dict.keys():
            merge_ing = "".join(ing_dict[key])
            contains_list = []
            for element in target_ing:
                contains_list.append(element in merge_ing)
            indicator_dict[key] = contains_list
            col_header = ["CONTAINS_%s" % ing for ing in target_ing]
            indicator_df = pd.DataFrame.from_dict(indicator_dict, orient = "index", columns = col_header)
    
    return indicator_df


def merge_indicator_df(split_regex, target_ing, df, match_all = True, ignore_organic = True):
    """
    Split ingredients specified by regex
    Appends indicator column each of the target ingredients in list
    Returns appended dataFrame
    """

    df_indicator = get_indicator_df(split_regex, target_ing, df, match_all, ignore_organic)
    df_indicator["ANY"] = df_indicator.any(axis = 1)
    df_indicator["ALL"] = df_indicator.all(axis = 1)
    df_merge = df.merge(df_indicator.reset_index().rename(columns = {"index" : "key"}))
    
    return df_merge


def get_flavors(split_regex, target_ing, df, criteria_any = True, match_all = True, ignore_organic = True):
    
    """
    Split ingredients specified by regex
    Returns dataFrame of only those that meet the critera
    Can set to either any or true (default: any)
    """
    
    df_merge = merge_indicator_df(split_regex, target_ing, df, match_all, ignore_organic)
    
    if criteria_any:
        df_filter = df_merge.loc[df_merge["ANY"] == True]
    else:
        df_filter = df_merge.loc[df_merge["ALL"] == True]
    
    return df_filter

In [235]:
split_regex = child_regex
ignore_organic = False
match_all = False
target_ing = "ORGANIC"

ing_dict = get_ingredients(split_regex, df, ignore_organic)
ing_list = list(get_ingredients_count(split_regex, df, ignore_organic).keys())

indicator_dict = {}

if match_all:
    
    target_ing_df = pd.DataFrame({"ingredients" : target_ing})
    target_ing = [ing.upper() for ing in target_ing if ing.upper() in ing_list]
    for key in ing_dict.keys():
        indicator_dict[key] = [item in ing_dict[key] for item in target_ing_df["ingredients"]]
        indicator_df = target_ing_df.join(pd.DataFrame(data = indicator_dict)).set_index("ingredients").transpose()


else:
    
    for key in ing_dict.keys():
        merge_ing = "".join(ing_dict[key])
        indicator_dict[key] = target_ing in merge_ing
        indicator_df = pd.DataFrame.from_dict(indicator_dict, orient = "index", columns = ["CONTAINS_%s" % (target_ing)])


In [144]:
#Example functions

target_ing = ["chocolate", "cream", "mango", "sodium"]
get_indicator_df(r'[.,:()]', target_ing, df)

,0,1,2
ingredients,CHOCOLATE,CREAM,MANGO
0_bj,False,True,False
1_bj,False,True,False
2_bj,False,True,False
3_bj,False,True,False
...,...,...,...
64_breyers,False,True,False
65_breyers,False,True,False
66_breyers,False,True,False
67_breyers,False,True,False


## Prepare indicator dataframes

In [145]:
parents_regex = r'[.,]\s*(?![^()]*\))' #Ignore parenthesis: i.e. Chocolate (Cocoa, Sugar, ...etc)
child_regex = r'[.,:()]' #Separates at parethesis: i.e. Chocolate, Cocoa, Sugar are considered separately

In [305]:
csv_path = os.path.join(os.getcwd(), "csv")

if not os.path.isdir(csv_path):
    os.mkdir(csv_path)

## Case 1: Dairy-Free Indicators

In [146]:
dairy_keywords = list(dairy[0].apply(lambda x: x.upper())) #From .csv file

dairy_df = get_indicator_df(child_regex, dairy_keywords, df) 

df_dairy_keys = list(get_flavors(child_regex, dairy_keywords, df)["key"])

dairy_free = df.loc[df["key"].isin(df_dairy_keys) == False]

In [147]:
dairy_free.name #Check flavor names

71                    Chocolate Fudge Non-Dairy Bar
73         Chocolate Salted Fudge Truffle Non-Dairy
75                        Coconut Caramel Non-Dairy
77     Coconut Caramel Dark Chocolate Non-Dairy Bar
91                                     Lemon Sorbet
94                                     Mango Sorbet
98          Peanut Butter Chocolate Fudge Non-Dairy
99      Peanut Butter Chocolate Fudge Non-Dairy Bar
104                                Raspberry Sorbet
127                         ALPHONSO MANGO SORBETTO
139                        COCONUT CHOCOLATE COOKIE
142                       COLD BREW COFFEE SORBETTO
144                         DARK CHOCOLATE SORBETTO
159                    PEANUT BUTTER FUDGE SORBETTO
164                        ROMAN RASPBERRY SORBETTO
168                    STRAWBERRY HIBISCUS SORBETTO
198                 Non-Dairy Vanilla Peanut Butter
222                 Non-Dairy OREO® Cookies & Cream
Name: name, dtype: object

This returns just the list of flavors that are dairy free.
We want to have the whole flavor list with dairy indicators

In [300]:
#Merge with original dataFrame, drop unnecessary columns.

dairy_indicator_df = merge_indicator_df(child_regex, dairy_keywords, df)

keep_cols = [
    "brand",
    "key",
    "name",
    "rating",
    "rating_count",
    "ANY"
]

dairy_cleaned = dairy_indicator_df[keep_cols].rename(columns = {"ANY" : "contains_dairy"})

#Export to csv

dairy_cleaned.to_csv(os.path.join(csv_path, "dairy_indicators.csv"))

dairy_cleaned.head()

,brand,key,name,rating,rating_count,contains_dairy
0,bj,0_bj,Salted Caramel Core,3.7,208,True
1,bj,1_bj,Netflix & Chilll'd™,4.0,127,True
2,bj,2_bj,Chip Happens,4.7,130,True
3,bj,3_bj,Cannoli,3.6,70,True
4,bj,4_bj,Gimme S’more!™,4.5,281,True


## Case 2: Organic Ice Cream

In [302]:
organic_indicator_df = merge_indicator_df(child_regex, "ORGANIC", df, match_all = False, ignore_organic = False)

keep_cols = [
    "brand",
    "key",
    "name",
    "rating",
    "rating_count",
    "CONTAINS_ORGANIC"
]

organic_cleaned = organic_indicator_df[keep_cols]

#Export to csv

organic_cleaned.to_csv(os.path.join(csv_path, "organic_indicators.csv"))

organic_cleaned.loc[organic_cleaned["CONTAINS_ORGANIC"] == True].head()

,brand,key,name,rating,rating_count,CONTAINS_ORGANIC
154,talenti,27_talenti,ORGANIC BROWN BUTTER CARAMEL,4.5,20,True
155,talenti,28_talenti,ORGANIC CHOCOLATE MOUSSE GELATO,4.3,13,True
156,talenti,29_talenti,ORGANIC GINGER MATCHA GELATO,4.7,21,True
157,talenti,30_talenti,ORGANIC OAK-AGED VANILLA GELATO,4.8,19,True


## Case 3: Contains Chocolate

In [303]:
chocolate_indicator_df = merge_indicator_df(child_regex, "CHOCOLATE", df, match_all = False)

keep_cols = [
    "brand",
    "key",
    "name",
    "rating",
    "rating_count",
    "CONTAINS_CHOCOLATE"
]

chocolate_cleaned = chocolate_indicator_df[keep_cols]

#Export to csv

chocolate_cleaned.to_csv(os.path.join(csv_path, "chocolate_indicators.csv"))

chocolate_cleaned.loc[chocolate_cleaned["CONTAINS_CHOCOLATE"] == True].head()

,brand,key,name,rating,rating_count,CONTAINS_CHOCOLATE
5,bj,5_bj,Peanut Butter Half Baked®,4.9,14,True
12,bj,12_bj,Brewed to Matter™,4.7,35,True
15,bj,15_bj,Chillin' the Roast™,3.3,172,True
16,bj,16_bj,Chocolate Chip Cookie Dough,4.6,983,True
19,bj,19_bj,Chocolate Therapy®,4.8,72,True


## Case 4: Contains Peanuts

In [299]:
peanut_indicator_df = merge_indicator_df(child_regex, "PEANUT", df, match_all = False)

keep_cols = [
    "brand",
    "key",
    "name",
    "rating",
    "rating_count",
    "CONTAINS_PEANUT"
]

peanut_cleaned = peanut_indicator_df[keep_cols]

#Export to csv

peanut_cleaned.to_csv(os.path.join(csv_path, "peanut_indicators.csv"))

peanut_cleaned.loc[peanut_cleaned["CONTAINS_PEANUT"] == True].head()

,brand,key,name,rating,rating_count,CONTAINS_PEANUT
1,bj,1_bj,Netflix & Chilll'd™,4.0,127,True
5,bj,5_bj,Peanut Butter Half Baked®,4.9,14,True
6,bj,6_bj,Berry Sweet Mascarpone,4.6,10,True
7,bj,7_bj,Chocolate Peanut Butter Split,5.0,7,True
11,bj,11_bj,Bourbon Pecan Pie,4.6,9,True


## Case 5:  Raspberries, Peanuts, Almonds, Coffee, Chocolate, Strawberries (Any)

In [304]:
target_ingredients = ["RASPBERRIES", "PEANUT", "ALMOND", "COFFEE", "CHOCOLATE", "STRAWBERRIES"]

any_indicator_df = merge_indicator_df(child_regex, target_ingredients, df, match_all = False)

contains_cols = ["CONTAINS_%s" % ing for ing in target_ingredients]

keep_cols = [
    "brand",
    "key",
    "name",
    "rating",
    "rating_count"
]

keep_cols.extend(contains_cols)
keep_cols.append("ANY")

any_cleaned = any_indicator_df[keep_cols]

#Export to csv

any_cleaned.to_csv(os.path.join(csv_path, "any_indicators.csv"))

any_cleaned.head()

,brand,key,name,rating,rating_count,CONTAINS_RASPBERRIES,CONTAINS_PEANUT,CONTAINS_ALMOND,CONTAINS_COFFEE,CONTAINS_CHOCOLATE,CONTAINS_STRAWBERRIES,ANY
0,bj,0_bj,Salted Caramel Core,3.7,208,False,False,False,False,False,False,False
1,bj,1_bj,Netflix & Chilll'd™,4.0,127,False,True,False,False,False,False,True
2,bj,2_bj,Chip Happens,4.7,130,False,False,False,False,False,False,False
3,bj,3_bj,Cannoli,3.6,70,False,False,False,False,False,False,False
4,bj,4_bj,Gimme S’more!™,4.5,281,False,False,False,False,False,False,False


## Case 6: Contains Eggs

In [307]:
egg_indicator_df = merge_indicator_df(child_regex, "EGG", df, match_all = False)

keep_cols = [
    "brand",
    "key",
    "name",
    "rating",
    "rating_count",
    "CONTAINS_EGG"
]

egg_cleaned = egg_indicator_df[keep_cols]

#Export to csv

egg_cleaned.to_csv(os.path.join(csv_path, "egg_indicators.csv"))

egg_cleaned.loc[egg_cleaned["CONTAINS_EGG"] == False].head()

,brand,key,name,rating,rating_count,CONTAINS_EGG
70,hd,13_hd,Chocolate Dark Chocolate Ice Cream Bar,4.5,22,False
71,hd,14_hd,Chocolate Fudge Non-Dairy Bar,5.0,22,False
73,hd,16_hd,Chocolate Salted Fudge Truffle Non-Dairy,4.8,97,False
75,hd,18_hd,Coconut Caramel Non-Dairy,4.9,11,False
77,hd,20_hd,Coconut Caramel Dark Chocolate Non-Dairy Bar,4.6,31,False
